[ Custom Model Class 정의 및 설계]
- 데이테셋 및 구현방법에 따른 모델 설계
- 부모클래스 : nn.Module
- 필수 오버라이딩 메서드 : 상속 받은 메서드 중 재정의하는 것
    *  _ _init_ _(self)   : 모델 구조/구성 요소 정의 
    * forward(self, data) : 전방향/순방향 학습 기능 메서드

[ IRIS Custom Model Class 정의 및 설계]
- 데이테셋 및 구현방법에 따른 모델 설계
- 부모클래스 : nn.Module
- 필수 오버라이딩 메서드 : 상속 받은 메서드 중 재정의하는 것
    *  _ _init_ _(self)   : 모델 구조/구성 요소 정의 
    * forward(self, data) : 전방향/순방향 학습 기능 메서드

[1] 모듈 로딩 <hr>

In [2]:
import torch                        # tensor, 내장함수 및 클래스 관련 모듈
import torch.nn as nn               # 인공신경망 관련 층, 함수 클래스 모듈
import torch.nn.functional as F     # 인공신경망 관련 층, AF, LF 함수 모듈

import pandas as pd                 # 데이터 로딩 및 전처리 관련 모듈 
import numpy as np                  # raw 데이터 즉, ndarray 데이터 관련 모듈


[2] 커스텀 모델 클래스 설계 <hr>

In [ ]:
## [2-1] 고정된 층과 퍼셉트론 구성 모델
## - 데 이  터 셋 : 피쳐 3 , 타겟 1 , 타겟 연속형
## - 학습    방법 : 지도학습 - 회귀
## - 구현알고리즘 : 인공신경망계열 DNN 
## -------------------------------------------------
##             입력       출력/퍼셉트론수    활성화함수
## - 입력층      3        """ 20                ReLU            => 각 개인이 알아서. 
## - 은닉층      20       10                ReLU                
## - 은닉층      10        5                ReLU
## - 출력층       5  """       1                 -
## -------------------------------------------------
""" 
데이터셋 iris: 피쳐 4, 타겟 1. 타겟은 범주형 3가지
학습 방법은 : 범주형이므로 분류 - 
구현알고리즘: 인공신경망 DNN
	  입력        퍼셉트론    활성화함수
입력층 4(피쳐4)   8(퍼셉트론)  RELU
은닉충  8           8 		RELU
은닉충  8           8 		RELU
은닉충  8           8 		RELU
출력층  8	    	3		
"""

class Cutom_iris_Model(nn.Module):
    # 인스턴스 초기화 및 생성 메서드
    def __init__(self):
        super().__init__() #부모(nn.Module)의 init 생성
        print('__init__()')
        self.in_layer=nn.Linear(4, 8)  #input, output
        self.hd_layer1=nn.Linear(8, 12)        # 히든1
        self.hd_layer2=nn.Linear(12, 8)         # 히든2
        self.hd_layer3=nn.Linear(8, 8)         # 히든3
        self.out_layer=nn.Linear(8, 3)

    # 순전파 학습 메서드 
    def forward(self, data):
        #- 입력층
        out=self.in_layer(data)      # data * weight + b
        out=F.relu(out)              # AF(data * weight + b) 결과물
                                     # 은닉으로 넘겨줌.

        #- 은닉층1
        out=self.hd_layer1(out)      # out * weight + b
        out=F.relu(out)              # AF(out * weight + b) 결과물        

        #- 은닉층2
        out=self.hd_layer2(out)      # out * weight + b
        out=F.relu(out)              # AF(out * weight + b) 결과물    

        #- 은닉층3
        out=self.hd_layer3(out)      # out * weight + b
        out=F.relu(out)              # AF(out * weight + b) 결과물    
        
        
        #- 출력층 : 회귀로 결과 그라드 반환
        out=self.out_layer(out)      # out * weight + b

        return out 
               

In [32]:
## - 모델 사용 -----------------------------------
## 모델 인스턴스 생성 
model=Cutom_iris_Model()

## 모델 구조 확인
print(model)

__init__()
Cutom_iris_Model(
  (in_layer): Linear(in_features=4, out_features=8, bias=True)
  (hd_layer1): Linear(in_features=8, out_features=12, bias=True)
  (hd_layer2): Linear(in_features=12, out_features=8, bias=True)
  (hd_layer3): Linear(in_features=8, out_features=8, bias=True)
  (out_layer): Linear(in_features=8, out_features=3, bias=True)
)


In [33]:
# data load
irisDF = pd.DataFrame(pd.read_csv('../_data/iris.csv'))
irisDF.head()
irisDF.columns

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')

In [34]:
irisDF.loc[:,['sepal.length', 'sepal.width', 'petal.length', 'petal.width']].values
iris_encodeDF = irisDF.copy()

from sklearn.preprocessing import  LabelEncoder
le = LabelEncoder()
iris_encodeDF['variety'] = le.fit_transform(iris_encodeDF['variety'])


In [35]:
## 모델 동작 테스트
data=torch.FloatTensor(irisDF.loc[:,['sepal.length', 'sepal.width', 'petal.length', 'petal.width']].values)
# data=torch.FloatTensor(iris_encodeDF.values)
print(data)

## 모델에 데이터 전달
pre=model(data)

tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000],
        [5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.4000, 1.4000, 0.3000],
        [5.0000, 3.4000, 1.5000, 0.2000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [4.9000, 3.1000, 1.5000, 0.1000],
        [5.4000, 3.7000, 1.5000, 0.2000],
        [4.8000, 3.4000, 1.6000, 0.2000],
        [4.8000, 3.0000, 1.4000, 0.1000],
        [4.3000, 3.0000, 1.1000, 0.1000],
        [5.8000, 4.0000, 1.2000, 0.2000],
        [5.7000, 4.4000, 1.5000, 0.4000],
        [5.4000, 3.9000, 1.3000, 0.4000],
        [5.1000, 3.5000, 1.4000, 0.3000],
        [5.7000, 3.8000, 1.7000, 0.3000],
        [5.1000, 3.8000, 1.5000, 0.3000],
        [5.4000, 3.4000, 1.7000, 0.2000],
        [5.1000, 3.7000, 1.5000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [5.1000, 3.3000, 1.7000, 0

In [36]:
# 모델에 파라미터들
for name, param in model.named_parameters():
    print(name, param)                  ## 위에서 입력한 퍼셉트론의 수 만큼 weight와 bias 반환.

in_layer.weight Parameter containing:
tensor([[-0.0932, -0.4788,  0.2629, -0.4959],
        [-0.2016, -0.3079,  0.0444, -0.0945],
        [-0.0106, -0.1653,  0.1312,  0.2776],
        [-0.4397,  0.0448,  0.2366, -0.4896],
        [-0.2978, -0.0135,  0.0841, -0.0417],
        [-0.3333, -0.4982,  0.4368, -0.0539],
        [ 0.0283,  0.0145, -0.3336, -0.4531],
        [ 0.1289, -0.2561,  0.2884, -0.2381]], requires_grad=True)
in_layer.bias Parameter containing:
tensor([ 0.1289, -0.4151,  0.1508,  0.3567, -0.3975,  0.1601,  0.0797,  0.3657],
       requires_grad=True)
hd_layer1.weight Parameter containing:
tensor([[ 0.0422, -0.1955,  0.2874,  0.1964, -0.0058, -0.0160, -0.1157, -0.0675],
        [ 0.1097, -0.2301,  0.3133,  0.1546,  0.2752,  0.2812, -0.2198,  0.0545],
        [ 0.2388, -0.2932, -0.0422, -0.2768, -0.0059, -0.0047,  0.1604,  0.3299],
        [ 0.3496, -0.0503, -0.2757,  0.1064,  0.2465, -0.2354, -0.0749,  0.2282],
        [ 0.1490, -0.1925,  0.0014,  0.1926, -0.2254,  0.1737,

In [37]:
model

Cutom_iris_Model(
  (in_layer): Linear(in_features=4, out_features=8, bias=True)
  (hd_layer1): Linear(in_features=8, out_features=12, bias=True)
  (hd_layer2): Linear(in_features=12, out_features=8, bias=True)
  (hd_layer3): Linear(in_features=8, out_features=8, bias=True)
  (out_layer): Linear(in_features=8, out_features=3, bias=True)
)